In [2]:
# 2024-09-02 测试, 可以跑通,loss-从5开始降低。

from mmcv.transforms import (LoadImageFromFile, RandomChoice,
                             RandomChoiceResize, RandomFlip)
from mmengine.config import read_base
from mmengine.optim.optimizer import OptimWrapper
from mmengine.optim.scheduler.lr_scheduler import LinearLR, PolyLR
from torch.nn.modules.batchnorm import SyncBatchNorm as SyncBN
from torch.optim import AdamW
from mmseg.datasets.transforms import (LoadAnnotations, PackSegInputs,
                                       PhotoMetricDistortion, RandomCrop,
                                       ResizeShortestEdge)
from mmseg.datasets.transforms.loading import LoadSingleRSImageFromFile
from mmseg.engine.optimizers import LayerDecayOptimizerConstructor


# EncoderDecoder
from mmseg.models.segmentors.encoder_decoder import EncoderDecoder
from mmseg.models.segmentors.atl_hiera_37_encoder_decoder import ATL_Hiera_EncoderDecoder
from mmseg.models.segmentors.atl_multi_encoder_multi_decoder import ATL_Multi_Encoder_Multi_Decoder
# SegDataPreProcessor
from mmseg.models.data_preprocessor import SegDataPreProcessor
# Backbone
from mmseg.models.backbones import BEiTAdapter
from mmseg.models.backbones import ViTAdapter
# DecodeHead
from mmseg.models.decode_heads.uper_head import UPerHead
from mmseg.models.decode_heads.atl_hiera_37_uper_head_multi_convseg import ATL_hiera_UPerHead_Multi_convseg
from mmseg.models.decode_heads.fcn_head import FCNHead
# Loss
from mmseg.models.losses.atl_hiera_37_loss import ATL_Hiera_Loss
from mmseg.models.losses.atl_hiera_37_loss_convseg import ATL_Hiera_Loss_convseg
from mmseg.models.losses.cross_entropy_loss import CrossEntropyLoss
# Evaluation
from mmseg.evaluation import IoUMetric

# with read_base():
#     from ..._base_.datasets.a_atl_0_paper_5b_GF2_19class import *
#     from ..._base_.default_runtime import *
#     # from ..._base_.models.upernet_beit_potsdam import *
#     from ..._base_.schedules.schedule_80k import *


In [47]:
pretrained = 'checkpoints/2-对比实验的权重/vit-adapter-offical/BEiT/beitv2_large_patch16_224_pt1k_ft21k-4chan.pth'

In [48]:
backbone_config = dict(
            type=BEiTAdapter,
            img_size=512,
            patch_size=16,
            embed_dim=768,  # B:768 L:1024
            in_channels=3,  
            depth=12,       # B:12 L:24
            num_heads=12,   # B:12 L:16
            deform_num_heads=12, # Adapter的参数： B:12 L:16
            mlp_ratio=4,
            qkv_bias=True,
            use_abs_pos_emb=False,
            use_rel_pos_bias=True,
            init_values=1e-6,
            drop_path_rate=0.3,
            conv_inplane=64,
            n_points=4,
            cffn_ratio=0.25,
            deform_ratio=0.5,
            with_cp=False,  # set with_cp=True to save memory
            # interaction_indexes=[[0, 5], [6, 11], [12, 17], [18, 23]], # large
            interaction_indexes=[[0, 2], [3, 5], [6, 8], [9, 11]],  # base
            init_cfg=dict(type='Pretrained', checkpoint=pretrained) # 不加预训练权重
        )  #backbone 完全一样

In [49]:
norm_cfg = dict(type=SyncBN, requires_grad=True)

L1_num_classes = 5  # number of L1 Level label   # 5
L2_num_classes = 10  # number of L1 Level label  # 11  5+11+21=37类
L3_num_classes = 19  # number of L1 Level label  # 21


decode_head_config=dict(
            type=UPerHead,
            # in_channels=[1024, 1024, 1024, 1024],  # 和vit的结构保持一致，large的话1024
            in_channels=[768, 768, 768, 768],  # 和vit的结构保持一致，large的话1024
            in_index=[0, 1, 2, 3],
            pool_scales=(1, 2, 3, 6),
            channels=768,   # 这是个 啥参数来着？
            dropout_ratio=0.1,
            num_classes=L3_num_classes,
            # num_classes_level_list=[5,10,19], #37
            norm_cfg=norm_cfg,
            align_corners=False,
            # loss_decode=dict(
            #     type=ATL_Hiera_Loss_convseg, num_classes=[5,10,19], loss_weight=1.0)),
            loss_decode=dict(
                type=CrossEntropyLoss, use_sigmoid=False, loss_weight=1.0))


In [53]:
backbone_config.update(in_channels=4)
backbone_config

{'type': mmseg.models.backbones.beit_adapter.BEiTAdapter,
 'img_size': 512,
 'patch_size': 16,
 'embed_dim': 768,
 'in_channels': 4,
 'depth': 12,
 'num_heads': 12,
 'deform_num_heads': 12,
 'mlp_ratio': 4,
 'qkv_bias': True,
 'use_abs_pos_emb': False,
 'use_rel_pos_bias': True,
 'init_values': 1e-06,
 'drop_path_rate': 0.3,
 'conv_inplane': 64,
 'n_points': 4,
 'cffn_ratio': 0.25,
 'deform_ratio': 0.5,
 'with_cp': False,
 'interaction_indexes': [[0, 2], [3, 5], [6, 8], [9, 11]],
 'init_cfg': {'type': 'Pretrained',
  'checkpoint': 'checkpoints/2-对比实验的权重/vit-adapter-offical/BEiT/beitv2_large_patch16_224_pt1k_ft21k-4chan.pth'}}

In [57]:
backbone_MSI_3chan={**backbone_config, 'in_channels': 3}
backbone_MSI_3chan

{'type': mmseg.models.backbones.beit_adapter.BEiTAdapter,
 'img_size': 512,
 'patch_size': 16,
 'embed_dim': 768,
 'in_channels': 3,
 'depth': 12,
 'num_heads': 12,
 'deform_num_heads': 12,
 'mlp_ratio': 4,
 'qkv_bias': True,
 'use_abs_pos_emb': False,
 'use_rel_pos_bias': True,
 'init_values': 1e-06,
 'drop_path_rate': 0.3,
 'conv_inplane': 64,
 'n_points': 4,
 'cffn_ratio': 0.25,
 'deform_ratio': 0.5,
 'with_cp': False,
 'interaction_indexes': [[0, 2], [3, 5], [6, 8], [9, 11]],
 'init_cfg': {'type': 'Pretrained',
  'checkpoint': 'checkpoints/2-对比实验的权重/vit-adapter-offical/BEiT/beitv2_large_patch16_224_pt1k_ft21k-4chan.pth'}}

In [58]:
backbone_MSI_3chan={**backbone_config, 'in_channels': 4}
backbone_MSI_3chan

{'type': mmseg.models.backbones.beit_adapter.BEiTAdapter,
 'img_size': 512,
 'patch_size': 16,
 'embed_dim': 768,
 'in_channels': 4,
 'depth': 12,
 'num_heads': 12,
 'deform_num_heads': 12,
 'mlp_ratio': 4,
 'qkv_bias': True,
 'use_abs_pos_emb': False,
 'use_rel_pos_bias': True,
 'init_values': 1e-06,
 'drop_path_rate': 0.3,
 'conv_inplane': 64,
 'n_points': 4,
 'cffn_ratio': 0.25,
 'deform_ratio': 0.5,
 'with_cp': False,
 'interaction_indexes': [[0, 2], [3, 5], [6, 8], [9, 11]],
 'init_cfg': {'type': 'Pretrained',
  'checkpoint': 'checkpoints/2-对比实验的权重/vit-adapter-offical/BEiT/beitv2_large_patch16_224_pt1k_ft21k-4chan.pth'}}

In [65]:
backbone_MSI_3chan = dict(**backbone_config, in_channels=10)

# 打印更新后的字典
print(backbone_MSI_3chan)

TypeError: dict() got multiple values for keyword argument 'in_channels'

In [12]:
backbone_MSI_3chan=
print(backbone_MSI_3chan)

None
